# 라이브러리 설치 및 로딩

In [1]:
# 라이브러리 및 헬퍼 모듈 로딩 셀
# - 한글 폰트 설정 및 Google Drive 연결(Colab 환경)
# - helper_utils.py, helper_c0z0c_dev.py 최신 버전 다운로드 및 import
# - 항상 importlib.reload로 헬퍼 모듈을 새로 읽어서 사용
# - from helper_utils import * / from helper_c0z0c_dev import *로 함수 직접 사용
# - 코드/경로/저장 로직은 헬퍼 함수(get_path_modeling 등)로 통일

import importlib
from urllib.request import urlretrieve

urlretrieve("https://raw.githubusercontent.com/c0z0c/jupyter_hangul/refs/heads/beta/helper_utils.py", "helper_utils.py")
import helper_utils as hu
from helper_utils import *

urlretrieve("https://raw.githubusercontent.com/c0z0c/jupyter_hangul/refs/heads/beta/helper_c0z0c_dev.py", "helper_c0z0c_dev.py")
import importlib
import helper_c0z0c_dev as helper

helper = importlib.reload(helper)
from helper_c0z0c_dev import *

hu = importlib.reload(hu)
from helper_utils import *


helper_utils.py loaded
🌐 https://c0z0c.github.io/jupyter_hangul
✅ 설정 완료: 한글 폰트, plt 전역 등록, pandas 확장, 캐시 기능
pd commit 저장 경로 = d:\GoogleDrive\codeit_ai_g2b_search\scripts\김명환
🌐 https://c0z0c.github.io/jupyter_hangul
✅ 설정 완료: 한글 폰트, plt 전역 등록, pandas 확장, 캐시 기능
pd commit 저장 경로 = d:\GoogleDrive\codeit_ai_g2b_search\scripts\김명환
helper_utils.py loaded


In [2]:
# 코랩 휴지통 비우기
# 1. Google Drive 인증
def clear_google_drive_trash():
    if IS_COLAB:
        # print("코랩 환경에서 Google Drive 휴지통 비우기를 시작합니다.")
        from google.colab import auth
        auth.authenticate_user()

        # 2. Google Drive API v3 서비스 빌드
        # 'drive', 'v3'은 Drive API와 버전을 지정합니다.
        from googleapiclient.discovery import build
        drive_service = build('drive', 'v3')

        # 3. 휴지통 비우기 명령어 실행
        # files().emptyTrash().execute() 메서드를 호출하여 휴지통을 영구 삭제합니다.
        drive_service.files().emptyTrash().execute()
        # print("Google Drive 휴지통을 성공적으로 비웠습니다.")

clear_google_drive_trash()

# 기본 라이브러리

In [3]:

# 기본 라이브러리

import warnings
# Warning 서브클래스만 필터링 가능
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning)

# --- Scikit-learn: 데이터 전처리, 모델, 평가 ---
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.datasets import (
    fetch_california_housing, load_iris, make_moons, make_circles,
    load_breast_cancer, load_wine
)
from sklearn import datasets
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.metrics import average_precision_score

# --- 기타 라이브러리 ---
from PIL import Image
from PIL import ImageFilter
from PIL import ImageDraw
import albumentations as A
import IPython.display
#from tqdm import tqdm
from tqdm.notebook import tqdm

# --- PyTorch: 딥러닝 관련 ---
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import argparse

from datasets import Dataset
from torch.utils.data import DataLoader
from torchvision.transforms import v2
from torchvision.datasets import CocoDetection
from torchvision.transforms import functional as TF
from torch.nn import CrossEntropyLoss
from collections import OrderedDict
from datasets import DatasetDict
from torch.utils.data import DataLoader
from torch.optim import AdamW
from datasets import load_dataset
from transformers import AutoTokenizer

from sklearn.utils.class_weight import compute_class_weight

# --- 기타 ---
import re
import os
import copy
import sys
import json
import math
import random
import yaml
import shutil
import pandas as pd
import numpy as np
import zipfile
from typing import Union, List, Optional, Tuple
from pathlib import Path

import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import pandas as pd
import seaborn as sns
from datetime import datetime
from datetime import timezone, timedelta
import pytz

from dataclasses import dataclass, asdict
import sqlite3
from transformers import (
    AutoModelForSequenceClassification,
    get_linear_schedule_with_warmup
)
from peft import LoraConfig, get_peft_model, TaskType
from sklearn.metrics import (
    precision_recall_fscore_support,
    confusion_matrix,
    classification_report
)
import wandb

_kst = pytz.timezone('Asia/Seoul')

# GPU 설정
_device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
_device_cpu = torch.device('cpu')

  # 재현 가능한 결과를 위해
np.random.seed(42)
torch.manual_seed(42)
if _device == 'cuda':
    torch.cuda.manual_seed_all(42)

logger = logging.getLogger(__name__)
#logger = logging.getLogger()
#logger.setLevel(logging.DEBUG)
if IS_COLAB:
  logger.setLevel(logging.INFO)
else:
  logger.setLevel(logging.DEBUG)
  
print(f"Python: {sys.version}")
print(f"PyTorch: {torch.__version__}")
print(f"CUDA Available: {torch.cuda.is_available()}")
logger.info(f"라이브러리 로드 완료 사용장치:{_device}")


2025-11-19 15:35:45 [I] 라이브러리 로드 완료 사용장치:cpu


Python: 3.10.19 | packaged by Anaconda, Inc. | (main, Oct 21 2025, 16:41:31) [MSC v.1929 64 bit (AMD64)]
PyTorch: 2.9.0+cpu
CUDA Available: False


In [4]:
# [OpenAI API 로그인 및 연결 테스트 셀]
# Colab: Colab 비밀에서 openai_api_key 로드
# 로컬: .env 파일에서 openai_api_key 로드
# API 키는 환경변수로만 처리 (노출 금지)

logger.setLevel(logging.DEBUG)

openai_api_key = None
if IS_COLAB:
    from google.colab import userdata
    openai_api_key = userdata.get('OPENAI_API_KEY')
else:
    from dotenv import load_dotenv
    load_dotenv(override=True)
    openai_api_key = os.getenv("OPENAI_API_KEY")

if openai_api_key:
    openai_api_key = openai_api_key.strip()
    os.environ["OPENAI_API_KEY"] = openai_api_key # LangChain
    logger.debug(f"OPENAI_API_KEY [{openai_api_key[:4]}****{openai_api_key[-4:]}] 환경변수 설정 완료")
    logger.info("OPENAI_API_KEY 설정")
else:
    logger.warning("openai_api_key가 설정되지 않아 OpenAI 로그인 생략됨")

2025-11-19 15:36:03 [D] OPENAI_API_KEY [sk-p****PAUA] 환경변수 설정 완료
2025-11-19 15:36:03 [I] OPENAI_API_KEY 설정


In [5]:
# [OpenAI 사용 가능 모델 자동 조회]
# client.models.list()로 계정별 접근 가능 모델 확인
# GPT-3 ~ GPT-5 계열만 필터링

logger.setLevel(logging.DEBUG)

openai_api_key = None
if IS_COLAB:
    from google.colab import userdata
    openai_api_key = userdata.get('OPENAI_API_KEY')
else:
    from dotenv import load_dotenv
    load_dotenv(override=True)
    openai_api_key = os.getenv("OPENAI_API_KEY")

if openai_api_key:
    openai_api_key = openai_api_key.strip()
    os.environ["OPENAI_API_KEY"] = openai_api_key
    logger.debug(f"OPENAI_API_KEY [{openai_api_key[:4]}****{openai_api_key[-4:]}] 설정")
    
    from openai import OpenAI
    client = OpenAI(api_key=openai_api_key)
    
    # 사용 가능한 모델 조회
    models = client.models.list()
    for model in models.data :
        logger.debug(f"사용 가능 모델: {model.id}")
    
    # GPT 계열 필터링 (embedding, whisper, tts, dall-e 제외)
    gpt_models = [
        model.id for model in models.data 
    ]
    
    # 정렬 (최신 모델 우선)
    gpt_models.sort(reverse=True)
    
    logger.info(f"사용 가능한 GPT 모델: {len(gpt_models)}개")
    for model_id in gpt_models:
        logger.debug(f"  - {model_id}")
    
    # 선호 모델 중 사용 가능한 것 확인
    preferred = ['gpt-5-nano','gpt-5-mini', 'gpt-4o-mini', 'gpt-4o', 'gpt-4-turbo', 'gpt-3.5-turbo']
    available_preferred = [m for m in preferred if m in gpt_models]
    
    if available_preferred:
        logger.info(f"권장 모델: {available_preferred[0]}")
    else:
        logger.warning("선호 모델 없음")
        
else:
    logger.warning("OPENAI_API_KEY 미설정")

2025-11-19 15:36:06 [D] OPENAI_API_KEY [sk-p****PAUA] 설정
2025-11-19 15:36:08 [D] 사용 가능 모델: gpt-5-mini
2025-11-19 15:36:08 [D] 사용 가능 모델: gpt-5
2025-11-19 15:36:08 [D] 사용 가능 모델: gpt-5-nano
2025-11-19 15:36:08 [D] 사용 가능 모델: text-embedding-3-small
2025-11-19 15:36:08 [I] 사용 가능한 GPT 모델: 4개
2025-11-19 15:36:08 [D]   - text-embedding-3-small
2025-11-19 15:36:08 [D]   - gpt-5-nano
2025-11-19 15:36:08 [D]   - gpt-5-mini
2025-11-19 15:36:08 [D]   - gpt-5
2025-11-19 15:36:08 [I] 권장 모델: gpt-5-nano


In [6]:
logger.setLevel(logging.DEBUG)

openai_api_key = None
if IS_COLAB:
    from google.colab import userdata
    openai_api_key = userdata.get('OPENAI_API_KEY')
else:
    from dotenv import load_dotenv
    load_dotenv(override=True)
    openai_api_key = os.getenv("OPENAI_API_KEY")

if openai_api_key:
    openai_api_key = openai_api_key.strip()
    os.environ["OPENAI_API_KEY"] = openai_api_key
    logger.debug(f"OPENAI_API_KEY 환경변수 설정 완료")

    from openai import OpenAI
    client = OpenAI(api_key=openai_api_key)

    model_id = "gpt-5-mini"
    #model_id = "gpt-4o-mini"
    
    params = {
        "model": model_id,
        "messages": [{"role": "user", "content": "너는 gpt인가?"}],
    }
    
    # GPT-5/4.1/o1 계열 파라미터
    if model_id.startswith(("gpt-5", "gpt-4.1", "o1")):
        params["max_completion_tokens"] = 500
    else:
        params["max_tokens"] = 500
        params["temperature"] = 0.7
    
    response = client.chat.completions.create(**params)
    
    # 응답 분석
    message = response.choices[0].message
    finish_reason = response.choices[0].finish_reason
    content = message.content or "[응답 없음]"
    
    logger.debug(f"finish_reason: {finish_reason}")
    logger.debug(f"content length: {len(content)}")
    print_dic_tree(response.usage.to_dict())
    
    print(f"질문: 너는 gpt인가? [{model_id}]")
    print(f"응답: {content}")
    
    if finish_reason == "length":
        logger.warning("토큰 제한으로 응답 잘림")
    
    logger.info(f"OpenAI API 연결 성공 (모델: {model_id})")
else:
    logger.warning("openai_api_key가 설정되지 않아 OpenAI 로그인 생략됨")

2025-11-19 15:36:21 [D] OPENAI_API_KEY 환경변수 설정 완료
2025-11-19 15:36:27 [D] finish_reason: stop
2025-11-19 15:36:27 [D] content length: 54
2025-11-19 15:36:27 [I] OpenAI API 연결 성공 (모델: gpt-5-mini)


├─ completion_tokens [int]: 229
├─ prompt_tokens [int]: 12
├─ total_tokens [int]: 241
├─ completion_tokens_details [dict]
│  ├─ accepted_prediction_tokens [int]: 0
│  ├─ audio_tokens [int]: 0
│  ├─ reasoning_tokens [int]: 192
│  ├─ rejected_prediction_tokens [int]: 0
├─ prompt_tokens_details [dict]
│  ├─ audio_tokens [int]: 0
│  ├─ cached_tokens [int]: 0
질문: 너는 gpt인가? [gpt-5-mini]
응답: 네, 저는 OpenAI의 GPT 계열 언어 모델 기반 AI 어시스턴트입니다. 어떻게 도와드릴까요?


In [7]:
import sys
from pathlib import Path
project_root = Path().cwd().parent.parent
sys.path.insert(0, str(project_root))


In [8]:
from src import config
importlib.reload(config)
from src.config import get_config, Config
config = get_config()
config.DOCUMENTS_DB_PATH = str(project_root / "data" / "documents.db")
config.EMBEDDINGS_DB_PATH = str(project_root / "data" / "embeddings.db")
config.CHAT_HISTORY_DB_PATH = str(project_root / "data" / "chat_history.db")
config.VECTORSTORE_PATH = str(project_root / "data" / "vectorstore")
config.MARKER_DUMP_PATH = str(project_root / "data" / "markers")
config.CONFIG_PATH = str("config.json")
OPENAI_API_KEY = config.OPENAI_API_KEY
config.OPENAI_API_KEY = None
print_dic_tree(config.to_dict())
config.save_to_json()
config.OPENAI_API_KEY = OPENAI_API_KEY
config.MARKER_DUMP_ENABLED = True

설정 검증 통과
├─ version [str]: 1.0.0
├─ OPENAI_API_KEY [NoneType]: None
├─ OPENAI_MODEL [str]: gpt-5-mini
├─ OPENAI_TEMPERATURE [float]: 0.0
├─ OPENAI_EMBEDDING_MODEL [str]: text-embedding-3-small
├─ OPENAI_TOKENIZER_MODEL [str]: gpt-4
├─ CHUNKING_MODE [str]: token
├─ CHUNK_SIZE [int]: 1500
├─ CHUNK_OVERLAP [int]: 300
├─ CHUNK_SEPARATORS [list]
│  ├─ [0] [str]: 


│  ├─ ... (6 items omitted)
│  ├─ [7] [str]: 
├─ SUMMARY_RATIO [float]: 0.2
├─ SUMMARY_OVERLAP_RATIO [float]: 0.1
├─ SUMMARY_MIN_LENGTH [int]: 100
├─ SIMILARITY_THRESHOLD [float]: 0.75
├─ TOP_K_SUMMARY [int]: 5
├─ TOP_K_FINAL [int]: 2
├─ SCORE_GAP_THRESHOLD [float]: 0.15
├─ EMBEDDING_BATCH_SIZE [int]: 100
├─ EMBEDDING_DIMENSION [int]: 1536
├─ DATA_PATH [str]: data
├─ DOCUMENTS_DB_PATH [str]: d:\GoogleDrive\codeit_ai_g2b_search\data\documents.db
├─ EMBEDDINGS_DB_PATH [str]: d:\GoogleDrive\codeit_ai_g2b_search\data\embeddings.db
├─ CHAT_HISTORY_DB_PATH [str]: d:\GoogleDrive\codeit_ai_g2b_search\data\chat_history.db
├─ VECTORSTORE_P

In [9]:
from src.db import documents_db
importlib.reload(documents_db)
from src.processors import document_processor
importlib.reload(document_processor)
from src.processors.document_processor import DocumentProcessor

proc_doc = DocumentProcessor(config=config)
proc_doc.docs_db.summary()

2025-11-19 15:37:39 [I] [DOCDB] - DocumentsDB 모듈이 로드되었습니다.


2025-11-19 15:37:39 [I] [DOCDB] - DocumentsDB 모듈이 로드되었습니다.
Consider using the pymupdf_layout package for a greatly improved page layout analysis.
2025-11-19 15:37:40 [I] [DOCDB] - DocumentsDB 모듈이 로드되었습니다.
2025-11-19 15:37:42 [I] [DOCDB] - DocumentsDB 모듈이 로드되었습니다.
2025-11-19 15:37:42 [I] [DOCDB] - DocumentsDB 모듈이 로드되었습니다.
2025-11-19 15:37:42 [I] [DOCP] - DocumentProcessor 초기화 완료 (DB: d:\GoogleDrive\codeit_ai_g2b_search\data\documents.db)
2025-11-19 15:37:42 [I] [DOCDB] - ================================================================================
2025-11-19 15:37:42 [I] [DOCDB] - 데이터베이스: d:\GoogleDrive\codeit_ai_g2b_search\data\documents.db
2025-11-19 15:37:42 [I] [DOCDB] - 총 테이블 수: 1
2025-11-19 15:37:42 [I] [DOCDB] - ================================================================================
2025-11-19 15:37:42 [I] [DOCDB] - 테이블: TB_DOCUMENTS
2025-11-19 15:37:42 [I] [DOCDB] -   - 컬럼 수: 8
2025-11-19 15:37:42 [I] [DOCDB] -   - 컬럼명: file_hash, chunk_index, file_name, total_pages,

In [10]:
zip_file_path = str(Path(get_path_temp()) / 'codeit_ai_g2b_search_data.zip')
unzip_file_path = zip_file_path + '.unzip'

files = [
    str(Path(unzip_file_path) / 'files' / r'기초과학연구원_2025년도 중이온가속기용 극저온시스템 운전 용역.pdf'),
    str(Path(unzip_file_path) / 'files' / r'서울시립대학교_[사전공개] 학업성취도 다차원 종단분석 통합시스템 1차.pdf'),
    str(Path(unzip_file_path) / 'files' / r'서울특별시_2024년 지도정보 플랫폼 및 전문활용 연계 시스템 고도화 용.pdf'),
]

for file_path in files:
    logger.debug(f"파일 읽기: {Path(file_path).exists()} {file_path}")
# PDF -> db
for file_path in files:
    proc_doc.process_pdf(file_path)    

2025-11-14 14:13:02 [D] 파일 읽기: True d:\temp\codeit_ai_g2b_search_data.zip.unzip\files\기초과학연구원_2025년도 중이온가속기용 극저온시스템 운전 용역.pdf
2025-11-14 14:13:02 [D] 파일 읽기: True d:\temp\codeit_ai_g2b_search_data.zip.unzip\files\서울시립대학교_[사전공개] 학업성취도 다차원 종단분석 통합시스템 1차.pdf
2025-11-14 14:13:02 [D] 파일 읽기: True d:\temp\codeit_ai_g2b_search_data.zip.unzip\files\서울특별시_2024년 지도정보 플랫폼 및 전문활용 연계 시스템 고도화 용.pdf


2025-11-14 14:13:02 [I] [DOCP] - PDF 처리 시작: 기초과학연구원_2025년도 중이온가속기용 극저온시스템 운전 용역.pdf
2025-11-14 14:13:02 [D] [DOCP] - 파일 해시: 56e5a8ae813dbddf...
2025-11-14 14:13:02 [D] [DOCDB] - 검색 완료: 1건 (hash 모드, 검색어: 56e5a8ae813dbddfe5d46b5e5b6c0ea55ba121950217d0ef0aa8a0821f80bcce)
2025-11-14 14:13:02 [I] [DOCP] - 이미 처리된 파일 (skip): 기초과학연구원_2025년도 중이온가속기용 극저온시스템 운전 용역.pdf, hash=56e5a8ae813dbddf...
2025-11-14 14:13:02 [I] [DOCP] - PDF 처리 시작: 서울시립대학교_[사전공개] 학업성취도 다차원 종단분석 통합시스템 1차.pdf
2025-11-14 14:13:02 [D] [DOCP] - 파일 해시: cd81e943ce28db81...
2025-11-14 14:13:02 [D] [DOCDB] - 검색 완료: 1건 (hash 모드, 검색어: cd81e943ce28db811cbad55454b5fd406b27d0660dbb2bcfe521cc05fc7b7576)
2025-11-14 14:13:02 [I] [DOCP] - 이미 처리된 파일 (skip): 서울시립대학교_[사전공개] 학업성취도 다차원 종단분석 통합시스템 1차.pdf, hash=cd81e943ce28db81...
2025-11-14 14:13:02 [I] [DOCP] - PDF 처리 시작: 서울특별시_2024년 지도정보 플랫폼 및 전문활용 연계 시스템 고도화 용.pdf
2025-11-14 14:13:02 [D] [DOCP] - 파일 해시: a6f6691e69197b5f...
2025-11-14 14:13:02 [D] [DOCDB] - 검색 완료: 1건 (hash 모드, 검색어: a6f6691e69197b

In [11]:
result = proc_doc.docs_db.execute_query(r"SELECT * FROM TB_DOCUMENTS")
print_dic_tree(result,limit_value_text=70)

├─ [0] [dict]
│  ├─ file_hash [str]: 56e5a8ae813dbddfe5d46b5e5b6c0ea55ba121950217d0ef0aa8a0821f80bcce
│  ├─ file_name [str]: 기초과학연구원_2025년도 중이온가속기용 극저온시스템 운전 용역.pdf
│  ├─ total_pages [int]: 49
│  ├─ file_size [int]: 642123
│  ├─ text_content [str]: --- [빈페이지] ---
--- 페이지 2 ---

|Col1|소속 및 직위|성 명|서 명|
|---|---|---|---|...
│  ├─ created_at [str]: 2025-11-14 11:05:52
│  ├─ updated_at [str]: 2025-11-14 11:05:52.576906
├─ ... (1 items omitted)
├─ [2] [dict]
│  ├─ file_hash [str]: a6f6691e69197b5f7cdfbc3947fad8f6158dfde29d2a82d6d6429386ffab1999
│  ├─ file_name [str]: 서울특별시_2024년 지도정보 플랫폼 및 전문활용 연계 시스템 고도화 용.pdf
│  ├─ total_pages [int]: 75
│  ├─ file_size [int]: 3194810
│  ├─ text_content [str]: --- 페이지 1 ---

#### 목 차

사업안내

1. 사업의 개요 - ···························...
│  ├─ created_at [str]: 2025-11-14 11:12:58
│  ├─ updated_at [str]: 2025-11-14 11:12:58.991500


In [10]:
result = proc_doc.docs_db.search_documents("56e5a8ae813dbddfe5d46b5e5b6c0ea55ba121950217d0ef0aa8a0821f80bcce")
print_dic_tree(result,limit_value_text=10)

2025-11-19 15:38:09 [D] [DOCDB] - 검색 완료: 1건 (hash 모드, 검색어: 56e5a8ae813dbddfe5d46b5e5b6c0ea55ba121950217d0ef0aa8a0821f80bcce)
├─ [0] [dict]
│  ├─ file_hash [str]: 56e5a8ae81...
│  ├─ chunk_index [int]: 0
│  ├─ file_name [str]: 기초과학연구원_20...
│  ├─ total_pages [int]: 49
│  ├─ file_size [int]: 642123
│  ├─ text_content [str]: --- [빈페이지]...
│  ├─ created_at [str]: 2025-11-19...
│  ├─ updated_at [str]: 2025-11-19...


In [13]:

result = proc_doc.docs_db.search_documents("기초과학연구원_2025년도 중이온가속기용 극저온시스템 운전 용역.pdf")
print_dic_tree(result,limit_value_text=10)

2025-11-14 14:13:02 [D] [DOCDB] - 검색 완료: 1건 (filename 모드, 검색어: 기초과학연구원_2025년도 중이온가속기용 극저온시스템 운전 용역.pdf)
├─ [0] [dict]
│  ├─ file_hash [str]: 56e5a8ae81...
│  ├─ file_name [str]: 기초과학연구원_20...
│  ├─ total_pages [int]: 49
│  ├─ file_size [int]: 642123
│  ├─ text_content [str]: --- [빈페이지]...
│  ├─ created_at [str]: 2025-11-14...
│  ├─ updated_at [str]: 2025-11-14...


In [14]:
result = proc_doc.docs_db.search_documents("없는파일.pdf")
print_dic_tree(result,limit_value_text=10)

2025-11-14 14:13:02 [D] [DOCDB] - 검색 완료: 0건 (filename 모드, 검색어: 없는파일.pdf)
└─ [list] (0 items)


In [12]:
from src.processors import embedding_processor
importlib.reload(embedding_processor)
from src.processors.embedding_processor import EmbeddingProcessor
proc_emb = EmbeddingProcessor(config=config)
proc_emb.docs_db.summary()
print('.' * 80)
proc_emb.vector_manager.summary()

2025-11-19 16:26:46 [I] [DOCDB] - DocumentsDB 모듈이 로드되었습니다.
2025-11-19 16:26:46 [I] [VC] - VectorStoreManager 초기화 완료 (model=text-embedding-3-small, path=d:\GoogleDrive\codeit_ai_g2b_search\data\vectorstore)
2025-11-19 16:26:46 [W] [VC] - FAISS 인덱스 미존재: d:\GoogleDrive\codeit_ai_g2b_search\data\vectorstore.faiss — 더미 인덱스 생성
2025-11-19 16:26:47 [I] [VC] - 더미 FAISS 인덱스 생성 완료: d:\GoogleDrive\codeit_ai_g2b_search\data\vectorstore
2025-11-19 16:26:47 [I] [EMBP] - EmbeddingProcessor 초기화 완료 (chunk_size=1500, overlap=300, model=text-embedding-3-small, faiss_path=d:\GoogleDrive\codeit_ai_g2b_search\data\vectorstore)
2025-11-19 16:26:47 [I] [EMBP] - ================================================================================
2025-11-19 16:26:47 [I] [EMBP] - docs_db와 vector_manager 동기화 시작
2025-11-19 16:26:47 [I] [EMBP] - ================================================================================
2025-11-19 16:26:47 [I] [EMBP] - docs_db 파일 수: 100
2025-11-19 16:26:47 [I] [EMBP] - vector_manag

In [16]:
# re.split 동작 검증
test_content = """시작
--- 페이지 1 ---
첫번째 내용
--- 페이지 2 ---
두번째 내용"""

splits = re.split(r'--- 페이지 (\d+) ---', test_content)
logger.debug(f"split 길이: {len(splits)}")
for idx, part in enumerate(splits):
    logger.debug(f"[{idx}]: {repr(part[:30])}")

2025-11-14 14:13:06 [D] split 길이: 5
2025-11-14 14:13:06 [D] [0]: '시작\n'
2025-11-14 14:13:06 [D] [1]: '1'
2025-11-14 14:13:06 [D] [2]: '\n첫번째 내용\n'
2025-11-14 14:13:06 [D] [3]: '2'
2025-11-14 14:13:06 [D] [4]: '\n두번째 내용'


In [17]:
# clean_markdown_text 확장 유닛 테스트 (20개 케이스)
from src.processors import embedding_processor
importlib.reload(embedding_processor)
from src.processors.embedding_processor import EmbeddingProcessor
proc_emb = EmbeddingProcessor(config=config)

test_cases = [
    {"name": "TC01_코드블록_3백틱_보호", "input": "```python\ndef test():\n    pass\n```", "expected_contains": ["def test()", "pass"], "expected_not_contains": ["XPROTECTED"]},
    {"name": "TC02_코드블록_4백틱_보호", "input": "````markdown\n```code\ninside\n```\n````", "expected_contains": ["```code", "inside"], "expected_not_contains": ["XPROTECTED"]},
    {"name": "TC03_수식블록_보호", "input": "$$\n\\frac{1}{2}\n$$", "expected_contains": ["\\frac{1}{2}"], "expected_not_contains": ["XPROTECTED"]},
    {"name": "TC04_인라인수식_보호", "input": "The formula is $E=mc^2$ here.", "expected_contains": ["$E=mc^2$"], "expected_not_contains": ["XPROTECTED"]},
    {"name": "TC05_페이지마커_보호", "input": "--- 페이지 1 ---\ntext\n--- 페이지 2 ---", "expected_contains": ["--- 페이지 1 ---", "--- 페이지 2 ---"], "expected_not_contains": ["XPROTECTED"]},
    {"name": "TC06_빈페이지마커_보호", "input": "--- [빈페이지] ---\n--- [오류페이지] ---", "expected_contains": ["--- [빈페이지] ---", "--- [오류페이지] ---"], "expected_not_contains": ["XPROTECTED"]},
    {"name": "TC07_HTML태그_제거", "input": "<div>content</div>", "expected_contains": ["content"], "expected_not_contains": ["<div>", "</div>"]},
    {"name": "TC08_링크_제거_텍스트유지", "input": "[링크텍스트](https://example.com)", "expected_contains": ["링크텍스트"], "expected_not_contains": ["https://example.com", "[", "]"]},
    {"name": "TC09_이미지_제거", "input": "![alt](image.png)", "expected_contains": [], "expected_not_contains": ["![", "image.png"]},
    {"name": "TC10_강조기호_제거", "input": "**굵게** *기울임* ~~취소선~~", "expected_contains": ["굵게", "기울임", "취소선"], "expected_not_contains": ["**", "~~"]},
    {"name": "TC11_코드안_강조기호_보호", "input": "```python\n# **not bold**\n__init__\n```", "expected_contains": ["**not bold**", "__init__"], "expected_not_contains": ["XPROTECTED"]},
    {"name": "TC12_복합_중첩케이스", "input": "**text** ```code\n$math$\n``` [link](url) --- 페이지 1 ---", "expected_contains": ["text", "$math$", "link", "--- 페이지 1 ---"], "expected_not_contains": ["**", "url", "XPROTECTED"]},
    
    {"name": "TC13_표_구분선_유지", "input": "| A | B |\n|---|---|\n| 1 | 2 |", "expected_contains": ["|", "---", "A", "B", "1", "2"], "expected_not_contains": []},
    {"name": "TC14_인용문_기호_제거", "input": "> 인용문\n> 두번째 줄", "expected_contains": ["인용문", "두번째 줄"], "expected_not_contains": [">"]},
    {"name": "TC15_리스트_기호_유지", "input": "- 항목1\n* 항목2\n1. 항목3", "expected_contains": ["- 항목1", "* 항목2", "1. 항목3"], "expected_not_contains": []},
    {"name": "TC16_헤더_텍스트_유지", "input": "# H1\n## H2\n### H3", "expected_contains": ["H1", "H2", "H3"], "expected_not_contains": ["#"]},
    {"name": "TC17_수평선_vs_페이지마커", "input": "---\n--- 페이지 1 ---\n---", "expected_contains": ["--- 페이지 1 ---"], "expected_not_contains": []},
    {"name": "TC18_중첩_강조_제거", "input": "**_굵은기울임_**", "expected_contains": ["굵은기울임"], "expected_not_contains": ["**", "_"]},
    {"name": "TC19_탈출문자_처리", "input": r"\*not emphasis\* \[not link\]", "expected_contains": ["not emphasis", "[not link]"], "expected_not_contains": [r"\*", r"\["]},
    {"name": "TC20_연속공백_정규화", "input": "text   with    spaces\n\n\n\nmultiple newlines", "expected_contains": ["text with spaces", "multiple newlines"], "expected_not_contains": ["   ", "\n\n\n"]},
    
    {"name": "TC21_순서리스트_번호_유지", "input": " 1. 서론\n2) 방법론\n3. 결과", "expected_contains": ["1. 서론", "2) 방법론", "3. 결과"], "expected_not_contains": []},
    {"name": "TC22_순서없는리스트_유지", "input": "- 항목A\n* 항목B\n+ 항목C", "expected_contains": ["- 항목A", "* 항목B", "+ 항목C"], "expected_not_contains": []},
    {"name": "TC23_라인시작_공백제거", "input": "   들여쓰기\n\t탭문자\n일반", "expected_contains": ["들여쓰기", "탭문자", "일반"], "expected_not_contains": ["   ", "\t"]},
    {"name": "TC24_강조중복_단일처리", "input": "_이탤릭_ __볼드__", "expected_contains": ["이탤릭", "볼드"], "expected_not_contains": ["_"]},
    {"name": "TC25_복합_공백리스트강조", "input": "  - **항목**\n   1. _내용_", "expected_contains": ["- 항목", "1. 내용"], "expected_not_contains": ["**", "_", "  ", "   "]},
]

passed = 0
failed = 0
failed_cases = []

logger.setLevel(logging.ERROR)

for tc in test_cases:
    result = proc_emb.clean_markdown_text(tc["input"])
    
    pass_contains = all(exp in result for exp in tc["expected_contains"])
    pass_not_contains = all(exp not in result for exp in tc["expected_not_contains"])
    
    if pass_contains and pass_not_contains:
        passed += 1
        status = "PASS"
    else:
        failed += 1
        status = "FAIL"
        failed_cases.append({
            "name": tc["name"],
            "input": tc["input"],
            "result": result,
            "failed_contains": [e for e in tc["expected_contains"] if e not in result],
            "failed_not_contains": [e for e in tc["expected_not_contains"] if e in result]
        })
    
    print(f"[{status}] {tc['name']}")

logger.setLevel(logging.INFO)

print("=" * 60)
print(f"통과율: {passed}/{len(test_cases)} ({passed/len(test_cases)*100:.1f}%)")
print("=" * 60)

if failed_cases:
    print("\n실패 케이스 상세:")
    for fc in failed_cases:
        print(f"\n{fc['name']}")
        print(f"입력: {repr(fc['input'][:80])}")
        print(f"결과: {repr(fc['result'][:80])}")
        if fc['failed_contains']:
            print(f"누락된 문자열: {fc['failed_contains']}")
        if fc['failed_not_contains']:
            print(f"제거되지 않은 문자열: {fc['failed_not_contains']}")

2025-11-14 14:13:07 [I] [DOCDB] - DocumentsDB 모듈이 로드되었습니다.
2025-11-14 14:13:08 [I] [VC] - VectorStoreManager 초기화 완료 (model=text-embedding-3-small, path=d:\GoogleDrive\codeit_ai_g2b_search\data\vectorstore)
2025-11-14 14:13:08 [I] [VC] - FAISS 인덱스 로드 완료: d:\GoogleDrive\codeit_ai_g2b_search\data\vectorstore (벡터 수: 356)
2025-11-14 14:13:08 [I] [VC] - chunk_map 구축 완료: 356개 매핑
2025-11-14 14:13:08 [I] [EMBP] - EmbeddingProcessor 초기화 완료 (chunk_size=1500, overlap=300, model=text-embedding-3-small, faiss_path=d:\GoogleDrive\codeit_ai_g2b_search\data\vectorstore)
2025-11-14 14:13:08 [I] [EMBP] - ================================================================================
2025-11-14 14:13:08 [I] [EMBP] - docs_db와 vector_manager 동기화 시작
2025-11-14 14:13:08 [I] [EMBP] - ================================================================================
2025-11-14 14:13:08 [I] [EMBP] - docs_db 파일 수: 3
2025-11-14 14:13:08 [I] [EMBP] - vector_manager 파일 수: 3
2025-11-14 14:13:08 [I] [EMBP] - docs_db에만 

In [18]:
# 실제 문서 데이터로 전처리 테스트

file_hash_list = proc_emb.docs_db.execute_query(
    "SELECT file_hash FROM TB_DOCUMENTS LIMIT 1"
)

if file_hash_list:
    test_file_hash = file_hash_list[0]['file_hash']
    docs = proc_emb.docs_db.search_documents(test_file_hash)
    
    if docs:
        print_dic_tree(docs, limit_value_text=20)
        print("." * 80,flush=True)
        text_content = docs[0]['text_content']
        logger.info(f"text_content={len(text_content)}")
        text_content_cleaned = proc_emb.clean_markdown_text(text_content)
        logger.info(f"text_content_cleaned={len(text_content_cleaned)}")
        
        page_pattern = r'--- 페이지 (\d+) ---'
        page_splits = re.split(page_pattern, text_content)
        logger.info(f"page_splits={len(page_splits)}")
        
        for i in range(1, len(page_splits), 2):
            page_num = int(page_splits[i])
            page_text = page_splits[i+1] if i+1 < len(page_splits) else ""
            
            # 빈페이지/오류페이지 스킵
            if '--- [빈페이지] ---' in page_text or '--- [오류페이지] ---' in page_text:
                logger.info(f"[{page_num}] len=empty ")
                continue
            
            cleaned_text = proc_emb.clean_markdown_text(page_text)
            # if cleaned_text.strip():
            #     logger.info(f"[{page_num}] len={len(cleaned_text)} ")
            # else:
            #     logger.info(f"[{page_num}] len={len(cleaned_text)} ")
        
    else:
        print("페이지 데이터가 없습니다.")
else:
    print("문서가 없습니다.")

2025-11-14 14:13:09 [D] [DOCDB] - 검색 완료: 1건 (hash 모드, 검색어: 56e5a8ae813dbddfe5d46b5e5b6c0ea55ba121950217d0ef0aa8a0821f80bcce)
├─ [0] [dict]
│  ├─ file_hash [str]: 56e5a8ae813dbddfe5d4...
│  ├─ file_name [str]: 기초과학연구원_2025년도 중이온가속...
│  ├─ total_pages [int]: 49
│  ├─ file_size [int]: 642123
│  ├─ text_content [str]: --- [빈페이지] ---
--- 페...
│  ├─ created_at [str]: 2025-11-14 11:05:52
│  ├─ updated_at [str]: 2025-11-14 11:05:52....
................................................................................


2025-11-14 14:13:09 [I] text_content=67441
2025-11-14 14:13:09 [I] text_content_cleaned=60884
2025-11-14 14:13:09 [I] page_splits=97


In [19]:
proc_emb.docs_db.summary()

2025-11-14 14:13:10 [I] [DOCDB] - ================================================================================
2025-11-14 14:13:10 [I] [DOCDB] - 데이터베이스: d:\GoogleDrive\codeit_ai_g2b_search\data\documents.db
2025-11-14 14:13:10 [I] [DOCDB] - 총 테이블 수: 1
2025-11-14 14:13:10 [I] [DOCDB] - ================================================================================
2025-11-14 14:13:10 [I] [DOCDB] - 테이블: TB_DOCUMENTS
2025-11-14 14:13:10 [I] [DOCDB] -   - 컬럼 수: 7
2025-11-14 14:13:10 [I] [DOCDB] -   - 컬럼명: file_hash, file_name, total_pages, file_size, text_content, created_at, updated_at
2025-11-14 14:13:10 [I] [DOCDB] -   - 레코드 수: 3
2025-11-14 14:13:10 [I] [DOCDB] - --------------------------------------------------------------------------------


In [20]:
proc_emb.vector_manager.summary()

2025-11-14 14:13:10 [I] [VC] - ================================================================================
2025-11-14 14:13:10 [I] [VC] - VectorStore Summary
2025-11-14 14:13:10 [I] [VC] - ================================================================================
2025-11-14 14:13:10 [I] [VC] - 벡터 수 (Vector Count)       : 356
2025-11-14 14:13:10 [I] [VC] - 청크 개수 (Chunk Count)       : 356
2025-11-14 14:13:10 [I] [VC] - 차원 (Dimension)              : 1536
2025-11-14 14:13:10 [I] [VC] - 인덱스 타입 (Index Type)      : IndexFlatL2
2025-11-14 14:13:10 [I] [VC] - 임베딩 모델 (Embedding Model) : text-embedding-3-small
2025-11-14 14:13:10 [I] [VC] - 파일 경로 (File Path)         : d:\GoogleDrive\codeit_ai_g2b_search\data\vectorstore
2025-11-14 14:13:10 [I] [VC] - 토탈 사이즈 (Total Size)      : 2.97 MB
2025-11-14 14:13:10 [I] [VC] - --------------------------------------------------------------------------------
2025-11-14 14:13:10 [I] [VC] - 파일별 벡터 분포 (총 3개 파일)
2025-11-14 14:13:10 [I] [VC] - ----------

In [21]:
context_texts = proc_emb.docs_db.get_documents_all()
print_dic_tree(context_texts, limit_value_text=10)

├─ [0] [dict]
│  ├─ file_hash [str]: a6f6691e69...
│  ├─ file_name [str]: 서울특별시_2024...
│  ├─ total_pages [int]: 75
│  ├─ file_size [int]: 3194810
│  ├─ text_content [str]: --- 페이지 1 ...
│  ├─ created_at [str]: 2025-11-14...
│  ├─ updated_at [str]: 2025-11-14...
├─ ... (1 items omitted)
├─ [2] [dict]
│  ├─ file_hash [str]: 56e5a8ae81...
│  ├─ file_name [str]: 기초과학연구원_20...
│  ├─ total_pages [int]: 49
│  ├─ file_size [int]: 642123
│  ├─ text_content [str]: --- [빈페이지]...
│  ├─ created_at [str]: 2025-11-14...
│  ├─ updated_at [str]: 2025-11-14...


In [22]:
# 실제 문서 데이터로 임베딩 생성 및 벡터스토어 저장 테스트
from src import config
importlib.reload(config)
from src.config import get_config, Config

config = get_config("config_test.json")
config.OPENAI_API_KEY = openai_api_key

from src.processors import embedding_processor
importlib.reload(embedding_processor)
from src.processors.embedding_processor import EmbeddingProcessor
proc_emb = EmbeddingProcessor(config=config)
proc_emb.vector_manager.load()
logger.info(f"해시 {proc_emb.vector_manager.calculate_embedding_config_hash(context_texts[0]['file_hash'])} 처리 시작")
proc_emb.process_document(context_texts[0]['file_hash'], api_key=config.OPENAI_API_KEY)
proc_emb.vector_manager.save()
proc_emb.vector_manager.summary()
result = proc_emb.vector_manager.get_by_metadata(file_hash=context_texts[0]['file_hash'])
#print_dic_tree(result, limit_value_text=10)
metadata = result[0][0].metadata
print_dic_tree(metadata, limit_value_text=70)

설정 파일 로드 완료: config_test.json
신규 옵션 자동 적용: OPENAI_API_KEY, _instance
설정 검증 통과
2025-11-14 14:13:10 [I] [DOCDB] - DocumentsDB 모듈이 로드되었습니다.
2025-11-14 14:13:11 [I] [VC] - VectorStoreManager 초기화 완료 (model=text-embedding-3-small, path=d:\GoogleDrive\codeit_ai_g2b_search\data\vectorstore)
2025-11-14 14:13:11 [I] [VC] - FAISS 인덱스 로드 완료: d:\GoogleDrive\codeit_ai_g2b_search\data\vectorstore (벡터 수: 356)
2025-11-14 14:13:11 [I] [VC] - chunk_map 구축 완료: 356개 매핑
2025-11-14 14:13:11 [I] [EMBP] - EmbeddingProcessor 초기화 완료 (chunk_size=600, overlap=200, model=text-embedding-3-small, faiss_path=d:\GoogleDrive\codeit_ai_g2b_search\data\vectorstore)
2025-11-14 14:13:11 [I] [EMBP] - ================================================================================
2025-11-14 14:13:11 [I] [EMBP] - docs_db와 vector_manager 동기화 시작
2025-11-14 14:13:11 [I] [EMBP] - ================================================================================
2025-11-14 14:13:11 [I] [EMBP] - docs_db 파일 수: 3
2025-11-14 14:13:11 [I

2025-11-14 14:13:39 [I] 해시 ce7171ebfa63cac8c4a31369902b548c865b71f9be0e0a4b29f07c2e42465363 처리 시작


2025-11-14 14:13:39 [I] [EMBP] - 임베딩 처리 시작: a6f6691e69197b5f...
2025-11-14 14:13:39 [D] [DOCDB] - 검색 완료: 1건 (hash 모드, 검색어: a6f6691e69197b5f7cdfbc3947fad8f6158dfde29d2a82d6d6429386ffab1999)
2025-11-14 14:13:39 [I] [EMBP] - 임베딩 스킵: 동일 설정 (file_hash=a6f6691e69197b5f..., existing_count=345, config_hash=ce7171eb...)
2025-11-14 14:13:39 [I] [VC] - FAISS 인덱스 저장 완료: d:\GoogleDrive\codeit_ai_g2b_search\data\vectorstore
2025-11-14 14:13:39 [I] [VC] - ================================================================================
2025-11-14 14:13:39 [I] [VC] - VectorStore Summary
2025-11-14 14:13:39 [I] [VC] - ================================================================================
2025-11-14 14:13:39 [I] [VC] - 벡터 수 (Vector Count)       : 897
2025-11-14 14:13:39 [I] [VC] - 청크 개수 (Chunk Count)       : 897
2025-11-14 14:13:39 [I] [VC] - 차원 (Dimension)              : 1536
2025-11-14 14:13:39 [I] [VC] - 인덱스 타입 (Index Type)      : IndexFlatL2
2025-11-14 14:13:39 [I] [VC] - 임베딩 모델 (Embedding 

In [23]:
# 실제 문서 데이터로 임베딩 생성 및 벡터스토어 저장 테스트 중복

from src import config
importlib.reload(config)
from src.config import get_config, Config

config = get_config("config_test.json")
config.OPENAI_API_KEY = openai_api_key

from src.processors import embedding_processor
importlib.reload(embedding_processor)
from src.processors.embedding_processor import EmbeddingProcessor
proc_emb = EmbeddingProcessor(config=config)
proc_emb.vector_manager.load()
logger.info(f"해시 {proc_emb.vector_manager.calculate_embedding_config_hash(context_texts[0]['file_hash'])} 처리 시작")
proc_emb.process_document(context_texts[0]['file_hash'], api_key=config.OPENAI_API_KEY)
proc_emb.vector_manager.save()
proc_emb.vector_manager.summary()
result = proc_emb.vector_manager.get_by_metadata(file_hash=context_texts[0]['file_hash'])
#print_dic_tree(result, limit_value_text=10)
metadata = result[0][0].metadata
print_dic_tree(metadata, limit_value_text=70)

설정 파일 로드 완료: config_test.json
신규 옵션 자동 적용: OPENAI_API_KEY, _instance
설정 검증 통과
2025-11-14 14:14:07 [I] [DOCDB] - DocumentsDB 모듈이 로드되었습니다.
2025-11-14 14:14:10 [I] [VC] - VectorStoreManager 초기화 완료 (model=text-embedding-3-small, path=d:\GoogleDrive\codeit_ai_g2b_search\data\vectorstore)
2025-11-14 14:14:10 [I] [VC] - FAISS 인덱스 로드 완료: d:\GoogleDrive\codeit_ai_g2b_search\data\vectorstore (벡터 수: 897)
2025-11-14 14:14:10 [I] [VC] - chunk_map 구축 완료: 897개 매핑
2025-11-14 14:14:10 [I] [EMBP] - EmbeddingProcessor 초기화 완료 (chunk_size=600, overlap=200, model=text-embedding-3-small, faiss_path=d:\GoogleDrive\codeit_ai_g2b_search\data\vectorstore)
2025-11-14 14:14:10 [I] [EMBP] - ================================================================================
2025-11-14 14:14:10 [I] [EMBP] - docs_db와 vector_manager 동기화 시작
2025-11-14 14:14:10 [I] [EMBP] - ================================================================================
2025-11-14 14:14:10 [I] [EMBP] - docs_db 파일 수: 3
2025-11-14 14:14:10 [I

2025-11-14 14:14:10 [I] 해시 ce7171ebfa63cac8c4a31369902b548c865b71f9be0e0a4b29f07c2e42465363 처리 시작


2025-11-14 14:14:10 [I] [EMBP] - 임베딩 처리 시작: a6f6691e69197b5f...
2025-11-14 14:14:10 [D] [DOCDB] - 검색 완료: 1건 (hash 모드, 검색어: a6f6691e69197b5f7cdfbc3947fad8f6158dfde29d2a82d6d6429386ffab1999)
2025-11-14 14:14:10 [I] [EMBP] - 임베딩 스킵: 동일 설정 (file_hash=a6f6691e69197b5f..., existing_count=345, config_hash=ce7171eb...)
2025-11-14 14:14:10 [I] [VC] - FAISS 인덱스 저장 완료: d:\GoogleDrive\codeit_ai_g2b_search\data\vectorstore
2025-11-14 14:14:10 [I] [VC] - ================================================================================
2025-11-14 14:14:10 [I] [VC] - VectorStore Summary
2025-11-14 14:14:10 [I] [VC] - ================================================================================
2025-11-14 14:14:10 [I] [VC] - 벡터 수 (Vector Count)       : 897
2025-11-14 14:14:10 [I] [VC] - 청크 개수 (Chunk Count)       : 897
2025-11-14 14:14:10 [I] [VC] - 차원 (Dimension)              : 1536
2025-11-14 14:14:10 [I] [VC] - 인덱스 타입 (Index Type)      : IndexFlatL2
2025-11-14 14:14:10 [I] [VC] - 임베딩 모델 (Embedding 

In [24]:
# 실제 문서 데이터로 임베딩 생성 및 벡터스토어 저장 옵션 변경

from src import config
importlib.reload(config)
from src.config import get_config, Config

config = get_config("config.json")
config.OPENAI_API_KEY = openai_api_key

from src import vectorstore
importlib.reload(vectorstore)
from src.vectorstore import VectorStoreManager

from src.processors import embedding_processor
importlib.reload(embedding_processor)
from src.processors.embedding_processor import EmbeddingProcessor
proc_emb = EmbeddingProcessor(config=config)
proc_emb.vector_manager.load()
logger.info(f"해시 {proc_emb.vector_manager.calculate_embedding_config_hash(context_texts[0]['file_hash'])} 처리 시작")
proc_emb.process_document(context_texts[0]['file_hash'], api_key=config.OPENAI_API_KEY)
proc_emb.vector_manager.save()
proc_emb.vector_manager.summary()
result = proc_emb.vector_manager.get_by_metadata(file_hash=context_texts[0]['file_hash'])
#print_dic_tree(result, limit_value_text=10)
metadata = result[0][0].metadata
print_dic_tree(metadata, limit_value_text=70)

설정 파일 로드 완료: config.json
신규 옵션 자동 적용: OPENAI_API_KEY, _instance
설정 검증 통과
2025-11-14 14:14:14 [I] [DOCDB] - DocumentsDB 모듈이 로드되었습니다.
2025-11-14 14:14:17 [I] [VC] - VectorStoreManager 초기화 완료 (model=text-embedding-3-small, path=d:\GoogleDrive\codeit_ai_g2b_search\data\vectorstore)
2025-11-14 14:14:17 [I] [VC] - FAISS 인덱스 로드 완료: d:\GoogleDrive\codeit_ai_g2b_search\data\vectorstore (벡터 수: 897)
2025-11-14 14:14:17 [I] [VC] - chunk_map 구축 완료: 897개 매핑
2025-11-14 14:14:17 [I] [EMBP] - EmbeddingProcessor 초기화 완료 (chunk_size=1500, overlap=300, model=text-embedding-3-small, faiss_path=d:\GoogleDrive\codeit_ai_g2b_search\data\vectorstore)
2025-11-14 14:14:17 [I] [EMBP] - ================================================================================
2025-11-14 14:14:17 [I] [EMBP] - docs_db와 vector_manager 동기화 시작
2025-11-14 14:14:17 [I] [EMBP] - ================================================================================
2025-11-14 14:14:17 [I] [EMBP] - docs_db 파일 수: 3
2025-11-14 14:14:17 [I] [E

2025-11-14 14:14:38 [I] 해시 5d4952b7766f24b5c63d24e050b07343222f09ffc542c73284d3866b26ccaff2 처리 시작


2025-11-14 14:14:38 [I] [EMBP] - 임베딩 처리 시작: a6f6691e69197b5f...
2025-11-14 14:14:38 [D] [DOCDB] - 검색 완료: 1건 (hash 모드, 검색어: a6f6691e69197b5f7cdfbc3947fad8f6158dfde29d2a82d6d6429386ffab1999)
2025-11-14 14:14:38 [I] [EMBP] - 임베딩 스킵: 동일 설정 (file_hash=a6f6691e69197b5f..., existing_count=129, config_hash=5d4952b7...)
2025-11-14 14:14:38 [I] [VC] - FAISS 인덱스 저장 완료: d:\GoogleDrive\codeit_ai_g2b_search\data\vectorstore
2025-11-14 14:14:38 [I] [VC] - ================================================================================
2025-11-14 14:14:38 [I] [VC] - VectorStore Summary
2025-11-14 14:14:38 [I] [VC] - ================================================================================
2025-11-14 14:14:38 [I] [VC] - 벡터 수 (Vector Count)       : 356
2025-11-14 14:14:38 [I] [VC] - 청크 개수 (Chunk Count)       : 356
2025-11-14 14:14:38 [I] [VC] - 차원 (Dimension)              : 1536
2025-11-14 14:14:38 [I] [VC] - 인덱스 타입 (Index Type)      : IndexFlatL2
2025-11-14 14:14:38 [I] [VC] - 임베딩 모델 (Embedding 

In [25]:
from src import config
importlib.reload(config)
from src.config import get_config, Config

config = get_config("config.json")
config.OPENAI_API_KEY = openai_api_key

from src.processors import embedding_processor
importlib.reload(embedding_processor)
from src.processors.embedding_processor import EmbeddingProcessor
proc_emb = EmbeddingProcessor(config=config)
proc_emb.vector_manager.summary()


설정 파일 로드 완료: config.json
신규 옵션 자동 적용: OPENAI_API_KEY, _instance
설정 검증 통과
2025-11-14 14:14:38 [I] [DOCDB] - DocumentsDB 모듈이 로드되었습니다.
2025-11-14 14:14:40 [I] [VC] - VectorStoreManager 초기화 완료 (model=text-embedding-3-small, path=d:\GoogleDrive\codeit_ai_g2b_search\data\vectorstore)
2025-11-14 14:14:40 [I] [VC] - FAISS 인덱스 로드 완료: d:\GoogleDrive\codeit_ai_g2b_search\data\vectorstore (벡터 수: 356)
2025-11-14 14:14:40 [I] [VC] - chunk_map 구축 완료: 356개 매핑
2025-11-14 14:14:40 [I] [EMBP] - EmbeddingProcessor 초기화 완료 (chunk_size=1500, overlap=300, model=text-embedding-3-small, faiss_path=d:\GoogleDrive\codeit_ai_g2b_search\data\vectorstore)
2025-11-14 14:14:40 [I] [EMBP] - ================================================================================
2025-11-14 14:14:40 [I] [EMBP] - docs_db와 vector_manager 동기화 시작
2025-11-14 14:14:40 [I] [EMBP] - ================================================================================
2025-11-14 14:14:40 [I] [EMBP] - docs_db 파일 수: 3
2025-11-14 14:14:40 [I] [E